In [ ]:
from encoders.cgan import *
from tensorflow.keras.models import load_model
from encoders.stn import Localization, BilinearInterpolation
from PIL import Image

In [ ]:
backgrounds = load_imgs(f"data/{cfg.TEST_DATASET}/test/background/")
objects = load_imgs(f"data/{cfg.TEST_DATASET}/test/foreground/")
depth = load_imgs(f"data/{cfg.TEST_DATASET}/test/depth/")
print('Loaded: ', backgrounds.shape, objects.shape, depth.shape)

data = [backgrounds, objects, depth]
dataset = preprocess_data(data)

In [ ]:
n_samples = 3
n_rows = 3

# Plot background images
for i in range(n_samples):
    plt.subplot(n_rows, n_samples, 1 + i)
    plt.axis('off')
    plt.imshow(backgrounds[i].astype('uint8'))

# Plot depth images
for i in range(n_samples):
    plt.subplot(n_rows, n_samples, 1 + n_samples + i)
    plt.axis('off')
    plt.imshow(depth[i].astype('uint8'))

# Plot objects images
for i in range(n_samples):
    plt.subplot(n_rows, n_samples, 1 + 2 * n_samples + i)
    plt.axis('off')
    plt.imshow(objects[i].astype('uint8'))

plt.show()

In [ ]:
# USE LINE BELOW IF YOU TRAINED WITH STN
# model = load_model('logs/cgan/models/g_model.h5', custom_objects={'Localization': Localization, 'BilinearInterpolation': BilinearInterpolation})
model = load_model(f'{cfg.MODEL_DIR}g_model_epoch{cfg.TEST_EPOCH}.h5')

# select a sample of input images
[backgrounds, objects, depth] = dataset
# generate a batch of fake samples
generated, _ = generate_fake_samples(model, backgrounds, objects, depth, 1)
# scale all pixels from [-1,1] to [0,1]
backgrounds = (backgrounds + 1) / 2.0
generated = (generated + 1) / 2.0
objects = (objects + 1) / 2.0
depth = (depth + 1) / 2.0

output_folder = f'{cfg.GEN_DIR}{cfg.TEST_DATASET}/'
if not os.path.exists(output_folder):
	os.makedirs(output_folder)
for i in range(generated.shape[0]):
    image = generated[i].numpy() * 255  # Convert TensorFlow tensor to NumPy array and then scale
    image = image.astype(np.uint8)
    img = Image.fromarray(image)
    img.save(f'{output_folder}/generated_image_{i}.png')

print(f'Saved {generated.shape[0]} generated images to {output_folder}')